# Monocle教程
在发育过程中，细胞对刺激作出反应，并在整个生命过程中，从一种功能“状态”过渡到另一种功能“状态”。不同状态的细胞表达不同的基因，产生蛋白质和代谢物的动态重复序列，从而完成它们的工作。当细胞在状态之间移动时，它们经历一个转录重组的过程，一些基因被沉默，另一些基因被激活。这些瞬时状态通常很难描述，因为在更稳定的端点状态之间纯化细胞可能是困难的或不可能的。单细胞RNA-Seq可以使我们在不需要纯化的情况下看到这些状态。然而，要做到这一点，我们必须确定每个cell在可能的状态范围内的位置。

Monocle介绍了利用RNA-Seq进行单细胞轨迹分析的策略。Monocle不是通过实验将细胞纯化成离散状态，而是使用一种算法来学习每个细胞必须经历的基因表达变化序列，作为动态生物学过程的一部分。一旦它了解了基因表达变化的整体“轨迹”，Monocle就可以将每个细胞置于轨迹中的适当位置。如果这个过程有多个结果，Monocle将建立一个“分支”轨迹。

## 安装Monocle
安装相关的依赖包以及R包“monocle3”，注意R语言的版本最好在4.0以上，参考https://www.jianshu.com/p/79886c1affe5

In [ ]:
BiocManager::install(c('BiocGenerics', 'DelayedArray', 'DelayedMatrixStats',
                      'limma', 'S4Vectors', 'SingleCellExperiment',
                      'SummarizedExperiment', 'batchelor'))

#install.packages("devtools")
devtools::install_github('cole-trapnell-lab/leidenbase')
devtools::install_github('cole-trapnell-lab/monocle3')

library(monocle3)
package.version(monocle3)
library(louvain)
library(dplyr)

## 1. 创建对象
创建对象一共需要三个文件：  
（1）expression_matrix ：一个行为基因，列为细胞的基因表达矩阵。  
（2）cell_metadata ：一个数据框，行为细胞，列是细胞的属性（比如细胞类型、培养环境、培养时间等）。  
（3）gene_metadata是一个数据框，行为feature信息（比如基因），列是基因属性（比如GC含量）。重要的是，这个数据框中必须有这么一列：gene_short_name，其中保存基因名（简称或Symbol标准名均可，用于作图）。  
三个文件要满足并且要满足：expression_matrix的行与gene_metadata的行对应；expression_matrix的列与cell_metadata的行对应。


In [1]:
#下载数据
expression_matrix = readRDS(url("http://staff.washington.edu/hpliner/data/packer_embryo_expression.rds"))
cell_metadata = readRDS(url("http://staff.washington.edu/hpliner/data/packer_embryo_colData.rds"))
gene_annotation = readRDS(url("http://staff.washington.edu/hpliner/data/packer_embryo_rowData.rds"))

# 创建CDS(Cell Data Set)对象
cds <- new_cell_data_set(expression_matrix,
                         cell_metadata = cell_metadata,
                         gene_metadata = gene_annotation)
cds

ERROR: Error in new_cell_data_set(expression_matrix, cell_metadata = cell_metadata, : 没有"new_cell_data_set"这个函数


## 2. 数据预处理
这一步就是对数据进行归一化、标准化，初步降维是使用PCA（针对标准的RNA-seq，需要指定计算的主成分数）还是LSA（Latent semantic analysis，针对ATAC-seq），这些都是为后面的聚类操作（tSNE、UMAP做准备）。  
降维方法method = c("PCA", "LSI")，默认是PCA；默认维度num_dim是50；初步降维前归一化norm_method 默认是log处理；设置降维方法是PCA时，自动进行标准化处理。

In [ ]:
# 设置100个主成分，因为细胞数太多(4万多个)，成分太少不足以代表整体
cds <- preprocess_cds(cds, num_dim = 100, residual_model_formula_str = "~ bg.300.loading + bg.400.loading + bg.500.1.loading + bg.500.2.loading + bg.r17.loading + bg.b01.loading + bg.b02.loading")

## 3. UMAP降维

In [ ]:
cds <- reduce_dimension(cds)
#查看降维后的效果
plot_cells(cds, label_groups_by_cluster=FALSE,  color_cells_by = "cell.type")

## 4. 看不同基因在不同分支的表达量 

In [ ]:
ciliated_genes = c("che-1",
                   "hlh-17",
                   "nhr-6",
                   "dmd-6",
                   "ceh-36",
                   "ham-1")
 
plot_cells(cds,
           genes=ciliated_genes,
           label_cell_groups=FALSE,
           show_trajectory_graph=FALSE)

## 5. 细胞聚类
Monocle不认为一组数据中的所有细胞都来自同一个”祖先“，很多实验中，它们会有多个发育轨迹。Monocle会通过聚类来判断细胞是否应该归属同一个发育轨迹。在monocle的的cluster_cells()中，细胞可以按cluster细分，还可以按partition归为大类。

In [ ]:
# 这里就用partition聚类
cds <- cluster_cells(cds)
plot_cells(cds, color_cells_by = "partition")

## 6. 寻找主路径
用learn graph在每个partition中寻找主路径。

In [ ]:
cds <- learn_graph(cds)
plot_cells(cds,
           color_cells_by = "cell.type",
           label_groups_by_cluster=FALSE,
           label_leaves=FALSE,
           label_branch_points=FALSE)

## 7. 对细胞出现的先后进行排序
也就是pseudotime（拟时序分析），推断哪个细胞先出来，哪个细胞后出来。它会对每个细胞在特定的过程（比如分化过程）中的参与度进行评估。

In [ ]:
plot_cells(cds,
           color_cells_by = "embryo.time.bin",
           label_cell_groups=FALSE,
           label_leaves=TRUE,
           label_branch_points=TRUE,
           graph_label_size=1.5)

图中的黑线就是整个架构（注意到整个图并非完全连接的，毕竟是按照partition聚类，每个partition中的细胞差异有点大）；浅灰色的圆圈表示”叶片“表示发育轨迹中的不同结局（可以认为拟时序分析的图是一个”根-茎-叶“结构），用参数label_leaves控制；黑色的圆圈表示分支节点，预示着其中的细胞会有不同发展方向，用参数label_branch_points控制；圈中数字大小表示出现时间的先后。  
	从上面的图中我们就能知道出现时间比较早的细胞位置，我们需要对全部的细胞都排个先后顺序，因此要用到order_cells() ，不过这个函数需要一个”根节点“位置，一般是一个partition分配一个根节点。可以通过get_earliest_principal_node()确定根节点：首先根据轨迹图中的节点将与它们最相近的细胞分成组，然后计算来自最早时间点的细胞所占组分，最后挑出包含早期细胞数量最多的节点，认为它是就是根节点。


In [ ]:
# 官方给出了一个函数，这里定义了一个time_bin，选择了最早的时间点区间。
get_earliest_principal_node <- function(cds, time_bin="130-170"){
  # 首先找到出现在最早时间区间的细胞ID
  cell_ids <- which(colData(cds)[, "embryo.time.bin"] == time_bin)
 
  closest_vertex <-
    cds@principal_graph_aux[["UMAP"]]$pr_graph_cell_proj_closest_vertex
  closest_vertex <- as.matrix(closest_vertex[colnames(cds), ])
  root_pr_nodes <-
    igraph::V(principal_graph(cds)[["UMAP"]])$name[as.numeric(names
      (which.max(table(closest_vertex[cell_ids,]))))]
 
  root_pr_nodes
}
cds = order_cells(cds, root_pr_nodes=get_earliest_principal_node(cds))

## 8. 可视化
对发育轨迹可视化

In [ ]:
plot_cells(cds,
           color_cells_by = "pseudotime",
           label_cell_groups=FALSE,
           label_leaves=FALSE,
           label_branch_points=FALSE,
           graph_label_size=1.5)

## 9. 3D发育轨迹

In [ ]:
#Working with 3D trajectories

cds_3d = reduce_dimension(cds, max_components = 3)
cds_3d = cluster_cells(cds_3d)
cds_3d = learn_graph(cds_3d)
cds_3d = order_cells(cds_3d, root_pr_nodes=get_earliest_principal_node(cds))

cds_3d_plot_obj = plot_cells_3d(cds_3d, color_cells_by="assigned_cell_type")
cds_3d_plot_obj

## 10. 差异分析

这一部分将会根据几种不同的标准进行差异分析，并且不同复杂程度的分析用时也有较大差别（几分钟甚至几小时）。文章只针对一小部分基因进行最简单的分析，但实际上它可以处理上千基因。

In [ ]:
#测试的小基因集
ciliated_genes = c("che-1",
                   "hlh-17",
                   "nhr-6",
                   "dmd-6",
                   "ceh-36",
                   "ham-1")
cds_subset = cds[rowData(cds)$gene_short_name %in% ciliated_genes,]

接下来，Monocle会对每个基因拟合一个回归模型，其中会加入实验中的一些因素（比如时间、处理等）。例如，在胚胎相关的单细胞数据中，细胞会在不同的时间点进行收集，这里就可以检测是否有基因在这段时间内的表达量发生了变化，利用fit_models。

In [ ]:
gene_fits = fit_models(cds_subset, model_formula_str = "~embryo.time")
# 其中model_formula_str就是要比较的分组对象，如果相获得不同的cluster或者partition的差异基因，就用model_formula_str = "~cluster"或者model_formula_str = "~partition"；另外还支持添加多个变量，比如考虑到批次效应 model_formula_str = "~embryo.time + batch"

然后看看哪些基因是与时间因素相关的

In [ ]:
fit_coefs = coefficient_table(gene_fits)
# 挑出时间相关的组分
emb_time_terms = fit_coefs %>% filter(term == "embryo.time")
# coefficient_table()默认使用 Benjamini and Hochberg（BH）方法进行了p值的校正，得到了q值
emb_time_terms %>% filter (q_value < 0.05) %>%
  select(gene_short_name, term, q_value, estimate)

画出小提琴图

In [ ]:
plot_genes_violin(cds_subset, group_cells_by="embryo.time.bin", ncol=2) +
    theme(axis.text.x=element_text(angle=45, hjust=1))

考虑批次效应

In [ ]:
gene_fits = fit_models(cds_subset, model_formula_str = "~embryo.time + batch")
fit_coefs = coefficient_table(gene_fits)
fit_coefs %>% filter(term != "(Intercept)") %>%
  select(gene_short_name, term, q_value, estimate)

模型评价

In [ ]:
gene_fits = fit_models(cds_subset, model_formula_str = "~embryo.time")
evaluate_fits(gene_fits)

使用compare_models()比较判断要不要考虑批次效应，结果会返回一个likelihood ratio test结果

In [ ]:
time_batch_models = fit_models(cds_subset,
                               model_formula_str = "~embryo.time + batch",
                               expression_family="negbinomial")
time_models = fit_models(cds_subset,
                        model_formula_str = "~embryo.time",
                        expression_family="negbinomial")
compare_models(time_batch_models, time_models) %>% select(gene_short_name, q_value)

## 参考文献
[1] Trapnell, C., et al. (2014). "The dynamics and regulators of cell fate decisions are revealed by pseudotemporal ordering of single cells." Nature Biotechnology 32(4): 381-386.